In [38]:
import pandas as pd
import numpy as np

In [39]:
pivot_data4 = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv')
activity_type_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/report_full_activity_type.csv')
smartphone = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s_split.csv')

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_2413/1207623996.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  smartphone = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s_split.csv')


In [40]:
def classify_motion_row(row):
    # Extract relevant data from the row
    accel_x = row['accel_x']
    accel_y = row['accel_y']
    accel_z = row['accel_z']
    gyro_x = row['gyro_x']
    gyro_y = row['gyro_y']
    gyro_z = row['gyro_z']

    # Calculate magnitudes along x, y, and z axes
    accel_mag_x_y = np.sqrt(accel_x**2 + accel_y**2)
    accel_mag_z = np.abs(accel_z)
    
    # Classify motion based on accelerometer and gyroscope readings
    if np.any((accel_x != 0) | (accel_y != 0) | (accel_z != 0)):
        # Check if all gyroscope readings are zero
        if not np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
            # Check if there is significant upward or downward motion along the z-axis
            if accel_z > 0 and accel_mag_x_y > accel_mag_z:
                motion_type = "Upward Horizontal Motion"
            elif accel_z < 0 and accel_mag_x_y > accel_mag_z:
                motion_type = "Downward Horizontal Motion"
            else:
                motion_type = "Other Moving"
        else:
            # If gyroscope readings are not all zero, classify as "Turning/Tilting"
            motion_type = "Turning/Tilting"
    elif np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
        # If accelerometer readings are all zero, check gyroscope readings
        motion_type = "Turning/Tilting"
    else:
        # If both accelerometer and gyroscope readings are zero, classify as "Stationary"
        motion_type = "Stationary"

    return pd.Series([motion_type])

# Apply the function row-wise to the dataframe
pivot_data4['motion_type'] = pivot_data4.apply(classify_motion_row, axis=1)


In [41]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Create and fit the LabelEncoder
label_encoder_motion = LabelEncoder()
pivot_data4['motion_encoded'] = label_encoder_motion.fit_transform(pivot_data4['motion_type'])

# Save the LabelEncoder to a file
joblib.dump(label_encoder_motion, '/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_motion.pkl')

# Get the encodings and decodings
encodings = label_encoder_motion.transform(label_encoder_motion.classes_)
decodings = label_encoder_motion.classes_

# Display the encodings and decodings
for encoding, decoding in zip(encodings, decodings):
    print(f"Encoding: {encoding}, Decoding: {decoding}")

Encoding: 0, Decoding: Downward Horizontal Motion
Encoding: 1, Decoding: Other Moving
Encoding: 2, Decoding: Stationary
Encoding: 3, Decoding: Turning/Tilting
Encoding: 4, Decoding: Upward Horizontal Motion


In [42]:
pivot_data4.fillna(0, inplace=True)

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import regularizers
import numpy as np

# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
y = pivot_data4['motion_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define the model with L1 regularization
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4['motion_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with class weights and early stopping
history = model.fit(X_train_scaled, y_train_encoded, epochs=20, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict, callbacks=[early_stopping])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/motion.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

2024-03-24 12:24:15.101925: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
3474/3474 [==============================] - 17s 4ms/step - loss: 1.3463 - accuracy: 0.8168 - val_loss: 0.5817 - val_accuracy: 0.9646
Epoch 2/20
3474/3474 [==============================] - 14s 4ms/step - loss: 0.7781 - accuracy: 0.9363 - val_loss: 0.4851 - val_accuracy: 0.9655
Epoch 3/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.6385 - accuracy: 0.9477 - val_loss: 0.4290 - val_accuracy: 0.9702
Epoch 4/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.5994 - accuracy: 0.9576 - val_loss: 0.4123 - val_accuracy: 0.9698
Epoch 5/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.5561 - accuracy: 0.9610 - val_loss: 0.4182 - val_accuracy: 0.9719
Epoch 6/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.5343 - accuracy: 0.9688 - val_loss: 0.3711 - val_accuracy: 0.9684
Epoch 7/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.5469 - accuracy: 0.9672 - val_loss: 0.3674 - val_ac

In [44]:
smartphone = smartphone.drop(columns = ['activity_type'])
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartphone['timestamp'] = pd.to_datetime(smartphone['timestamp'])
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartphone['timestamp'] = pd.to_datetime(smartphone['timestamp'])

# Group activity_type_full by timestamp and transform it to align with smartphone
activity_type_grouped = activity_type_full.groupby(smartphone['timestamp'])['activity_type'].apply(lambda x: ','.join(x)).reset_index()

# Merge activity_type_grouped with smartphone based on timestamp
smartphone = pd.merge(smartphone, activity_type_grouped, on='timestamp', how='left')

# Print or further process the updated DataFrame
print(smartphone)

smartphone.fillna(0, inplace=True)

# Filter the DataFrame to keep rows where "activity_type" is not equal to 0
merged_data_s_f = smartphone[smartphone['activity_type'] != 0]
# Assuming 'pivot_data4' is your DataFrame containing the timestamp column
merged_data_s_f['timestamp'] = pd.to_datetime(merged_data_s_f['timestamp'])

# Extract hour and minute from the timestamp and convert to minutes
merged_data_s_f['timestamp'] = merged_data_s_f['timestamp'].dt.hour * 60 + merged_data_s_f['timestamp'].dt.minute

merged_data_s_f_filtered = merged_data_s_f[['timestamp', 'activity_type']]

                     timestamp  step_detector_values  audio_value_1  \
0      2017-06-29 07:45:16.506                   1.0            0.0   
1      2017-06-29 07:45:17.407                   0.0            0.0   
2      2017-06-29 07:45:18.407                   0.0          227.0   
3      2017-06-29 07:45:19.407                   0.0          590.0   
4      2017-06-29 07:45:20.446                   0.0         1724.0   
...                        ...                   ...            ...   
617948 2017-07-13 20:13:33.671                   0.0         1956.0   
617949 2017-07-13 20:13:34.663                   0.0         2673.0   
617950 2017-07-13 20:13:35.663                   0.0         2340.0   
617951 2017-07-13 20:13:36.668                   0.0         2046.0   
617952 2017-07-13 20:13:37.668                   0.0         2441.0   

        audio_value_2  audio_value_3  audio_value_4 activity_type  
0             0.00000            0.0            0.0   Video games  
1          

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_2413/4048118510.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_s_f['timestamp'] = pd.to_datetime(merged_data_s_f['timestamp'])
/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_2413/4048118510.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_s_f['timestamp'] = merged_data_s_f['timestamp'].dt.hour * 60 + merged_data_s_f['timestamp'].dt.minute


In [45]:
# Assuming 'pivot_data4' is your DataFrame containing the timestamp column
pivot_data4['timestamp'] = pd.to_datetime(pivot_data4['timestamp'])
# Extract hour and minute from the timestamp and convert to minutes
pivot_data4['timestamp'] = pivot_data4['timestamp'].dt.hour * 60 + pivot_data4['timestamp'].dt.minute
# Merge the dataframes on the 'timestamp' column
pivot_data4_merged = pivot_data4.merge(merged_data_s_f_filtered, on='timestamp', how='left')
pivot_data4_merged.fillna(0, inplace=True)
pivot_data4_merged_filtered = pivot_data4_merged[pivot_data4_merged['activity_type'] != 0]

In [46]:
from sklearn.preprocessing import LabelEncoder

# Create and fit the LabelEncoder
label_encoder_activities = LabelEncoder()
pivot_data4_merged_filtered['activity_encoded'] = label_encoder_activities.fit_transform(pivot_data4_merged_filtered['activity_type'])

# Save the LabelEncoder to a file
joblib.dump(label_encoder_activities, '/Users/emilkoch/Desktop/2Tango/messenger/Research/Job_Library/label_encoder_activities.pkl')

# Get unique classes from the LabelEncoder
unique_classes = label_encoder_activities.classes_

# Print unique classes
print("Unique Classes:")
for class_ in unique_classes:
    print(class_)

# Get the encodings and decodings
encodings = label_encoder_activities.transform(label_encoder_activities.classes_)
decodings = label_encoder_activities.classes_

# Display the encodings and decodings
for encoding, decoding in zip(encodings, decodings):
    print(f"Encoding: {encoding}, Decoding: {decoding}")

Unique Classes:
At home
Cooking
Eat
Football
In bus
In computer
In vehicle
Meeting
Movie
On bus stop
Pause
Phone was out of the pocket (forgot)
Picnic 
Running
Shop
Shopping& wearing
Sleep
Took off glasses
Train
Video games
Walk
Walking&party
Watching TV
Work
Encoding: 0, Decoding: At home
Encoding: 1, Decoding: Cooking
Encoding: 2, Decoding: Eat
Encoding: 3, Decoding: Football
Encoding: 4, Decoding: In bus
Encoding: 5, Decoding: In computer
Encoding: 6, Decoding: In vehicle
Encoding: 7, Decoding: Meeting
Encoding: 8, Decoding: Movie
Encoding: 9, Decoding: On bus stop
Encoding: 10, Decoding: Pause
Encoding: 11, Decoding: Phone was out of the pocket (forgot)
Encoding: 12, Decoding: Picnic 
Encoding: 13, Decoding: Running
Encoding: 14, Decoding: Shop
Encoding: 15, Decoding: Shopping& wearing
Encoding: 16, Decoding: Sleep
Encoding: 17, Decoding: Took off glasses
Encoding: 18, Decoding: Train
Encoding: 19, Decoding: Video games
Encoding: 20, Decoding: Walk
Encoding: 21, Decoding: Walking&p

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_2413/3103381586.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['activity_encoded'] = label_encoder_activities.fit_transform(pivot_data4_merged_filtered['activity_type'])


In [125]:
def classify_activity(row):
    motion_type = row['motion_type']
    if motion_type == "Downward Horizontal Motion":
        return [13, 20, 21, 3, 6, 18, 4]
    elif motion_type == "Upward Horizontal Motion":
        return [6, 18, 4, 13, 20, 21, 3]
    elif motion_type == "Other Moving":
        return [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]
    elif motion_type == "Turning/Tilting":
        return [11, 17]
    elif motion_type == "Stationary":
        return [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]

# Apply the function to create the predicted_activity column
pivot_data4['predicted_activity'] = pivot_data4.apply(classify_activity, axis=1)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Define the classification function
def classify_activity(row):
    motion_type = row['motion_type']
    activities = {
        "Downward Horizontal Motion": [13, 20, 21, 3, 6, 18, 4],
        "Upward Horizontal Motion": [6, 18, 4, 13, 20, 21, 3],
        "Other Moving": [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3],
        "Turning/Tilting": [11, 17],
        "Stationary": [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]
    }
    activity_labels = []
    for label in range(24):  # Assuming there are 24 possible activity labels
        if any(label in activities[motion_type] for motion_type in activities):
            activity_labels.append(1)
        else:
            activity_labels.append(0)
    return activity_labels

# Apply the function to create the predicted activity column
pivot_data4['predicted_activity'] = pivot_data4.apply(classify_activity, axis=1)

# Convert predicted_activity to binary vectors
activity_df = pd.DataFrame(pivot_data4['predicted_activity'].tolist(), columns=[f'activity_{i}' for i in range(24)])
X = pd.concat([pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']], activity_df], axis=1)
y = pivot_data4['motion_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4['motion_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2, callbacks=[early_stopping])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/motion.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

In [111]:
pivot_data4

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,motion_encoded,predicted_activity,true_motion,activity_encoded
0,465,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,3
1,466,0.0,0.0,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Other Moving,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,15
2,466,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,8
3,466,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,8
4,466,0.0,0.0,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,...,0.0,0.0,0.0,0.0,0.0,Turning/Tilting,3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173657,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,17
173658,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,17
173659,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,17
173660,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,17


In [30]:
def classify_motion_activity(row):
    activity_encoded = row['activity_encoded']
    if activity_encoded in [13, 20, 21, 3, 6, 18, 4]:
        true_motion = 0
    elif activity_encoded in [6, 18, 4, 13, 20, 21, 3]:
        true_motion = 4
    elif activity_encoded in [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]:
        true_motion = 1
    elif activity_encoded in [11, 17]:
        true_motion = 3
    elif activity_encoded in [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]:
        true_motion = 2
    return true_motion

# Assuming 'pivot_data4' is your DataFrame
pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_2413/179481233.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)


In [50]:
pivot_data4_merged_filtered.fillna(0, inplace=True)

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_2413/202448761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered.fillna(0, inplace=True)


In [51]:
# Check for NaN values in X
nan_values = pivot_data4_merged_filtered.isna().any()


In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Define the classification function for motion activity
def classify_motion_activity(row):
    activity_encoded = row['activity_encoded']
    if activity_encoded in [13, 20, 21, 3, 6, 18, 4]:
        true_motion = 0
    elif activity_encoded in [6, 18, 4, 13, 20, 21, 3]:
        true_motion = 4
    elif activity_encoded in [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]:
        true_motion = 1
    elif activity_encoded in [11, 17]:
        true_motion = 3
    elif activity_encoded in [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]:
        true_motion = 2
    return true_motion

# Apply the function to create the 'true_motion' column
pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)

# Split the data into features (X) and target variable (y)
X = pivot_data4_merged_filtered[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'true_motion', 'timestamp']]
y = pivot_data4_merged_filtered['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4_merged_filtered['activity_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2, callbacks=[early_stopping])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_2413/963641440.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)


Epoch 1/20
246/246 [==============================] - 4s 5ms/step - loss: 2.7410 - accuracy: 0.1967 - val_loss: 2.0258 - val_accuracy: 0.3021
Epoch 2/20
246/246 [==============================] - 1s 4ms/step - loss: 2.0659 - accuracy: 0.2805 - val_loss: 1.7399 - val_accuracy: 0.3647
Epoch 3/20
246/246 [==============================] - 1s 4ms/step - loss: 1.8653 - accuracy: 0.3034 - val_loss: 1.6429 - val_accuracy: 0.3444
Epoch 4/20
246/246 [==============================] - 1s 5ms/step - loss: 1.7665 - accuracy: 0.3123 - val_loss: 1.5979 - val_accuracy: 0.3662
Epoch 5/20
246/246 [==============================] - 1s 4ms/step - loss: 1.7070 - accuracy: 0.3272 - val_loss: 1.5726 - val_accuracy: 0.3657
Epoch 6/20
246/246 [==============================] - 1s 5ms/step - loss: 1.6688 - accuracy: 0.3274 - val_loss: 1.5495 - val_accuracy: 0.3433
Epoch 7/20
246/246 [==============================] - 1s 5ms/step - loss: 1.6406 - accuracy: 0.3268 - val_loss: 1.5319 - val_accuracy: 0.3688
Epoch 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

The pipeline seems to function well. Much better than previously when using the same model for predicting the multiple possible activity classes and single-true classes. 

Upon reading, CNN model architecture seems to be used often for sensor readings. Let's test. 

In [66]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Define the classification function for motion activity
def classify_motion_activity(row):
    activity_encoded = row['activity_encoded']
    if activity_encoded in [13, 20, 21, 3, 6, 18, 4]:
        true_motion = 0
    elif activity_encoded in [6, 18, 4, 13, 20, 21, 3]:
        true_motion = 4
    elif activity_encoded in [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]:
        true_motion = 1
    elif activity_encoded in [11, 17]:
        true_motion = 3
    elif activity_encoded in [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]:
        true_motion = 2
    return true_motion

# Apply the function to create the 'true_motion' column
pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)

# Split the data into features (X) and target variable (y)
X = pivot_data4_merged_filtered[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'true_motion', 'timestamp']]
y = pivot_data4_merged_filtered['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4_merged_filtered['activity_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Encode class labels for the new target variable
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2, class_weight=class_weight_dict, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_2413/939017988.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)


Epoch 1/20
246/246 [==============================] - 4s 5ms/step - loss: 3.8787 - accuracy: 0.0709 - val_loss: 2.9474 - val_accuracy: 0.0504
Epoch 2/20
246/246 [==============================] - 1s 5ms/step - loss: 3.0744 - accuracy: 0.1118 - val_loss: 2.6874 - val_accuracy: 0.0839
Epoch 3/20
246/246 [==============================] - 1s 4ms/step - loss: 2.9214 - accuracy: 0.1196 - val_loss: 2.5109 - val_accuracy: 0.1226
Epoch 4/20
246/246 [==============================] - 1s 5ms/step - loss: 2.5372 - accuracy: 0.1183 - val_loss: 2.4462 - val_accuracy: 0.1089
Epoch 5/20
246/246 [==============================] - 1s 4ms/step - loss: 2.7070 - accuracy: 0.1223 - val_loss: 2.3393 - val_accuracy: 0.1099
Epoch 6/20
246/246 [==============================] - 1s 4ms/step - loss: 2.7376 - accuracy: 0.1256 - val_loss: 2.3005 - val_accuracy: 0.1134
Epoch 7/20
246/246 [==============================] - 1s 5ms/step - loss: 2.3783 - accuracy: 0.1201 - val_loss: 2.2742 - val_accuracy: 0.1175
Epoch 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

In [106]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Define the classification function for motion activity
def classify_motion_activity(row):
    activity_encoded = row['activity_encoded']
    if activity_encoded in [13, 20, 21, 3, 6, 18, 4]:
        true_motion = 0
    elif activity_encoded in [6, 18, 4, 13, 20, 21, 3]:
        true_motion = 4
    elif activity_encoded in [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]:
        true_motion = 1
    elif activity_encoded in [11, 17]:
        true_motion = 3
    elif activity_encoded in [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]:
        true_motion = 2
    return true_motion

# Apply the function to create the 'true_motion' column
pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)

# Split the data into features (X) and target variable (y)
X = pivot_data4_merged_filtered[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'true_motion', 'timestamp']]
y = pivot_data4_merged_filtered['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the data for compatibility with CNN
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Define the CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(pivot_data4_merged_filtered['activity_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Encode class labels for the new target variable
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=20, validation_split=0.2, class_weight=class_weight_dict, callbacks=[early_stopping])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_2413/3157097494.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)


Epoch 1/20
246/246 [==============================] - 3s 5ms/step - loss: 3.0462 - accuracy: 0.1047 - val_loss: 2.5486 - val_accuracy: 0.0641
Epoch 2/20
246/246 [==============================] - 1s 4ms/step - loss: 2.7464 - accuracy: 0.1439 - val_loss: 2.2695 - val_accuracy: 0.1419
Epoch 3/20
246/246 [==============================] - 1s 4ms/step - loss: 2.5809 - accuracy: 0.1388 - val_loss: 2.1659 - val_accuracy: 0.1267
Epoch 4/20
246/246 [==============================] - 1s 6ms/step - loss: 2.3109 - accuracy: 0.1408 - val_loss: 2.0643 - val_accuracy: 0.1399
Epoch 5/20
246/246 [==============================] - 1s 4ms/step - loss: 2.2527 - accuracy: 0.1344 - val_loss: 1.9805 - val_accuracy: 0.1083
Epoch 6/20
246/246 [==============================] - 1s 4ms/step - loss: 2.1700 - accuracy: 0.1341 - val_loss: 2.0296 - val_accuracy: 0.0855
Epoch 7/20
246/246 [==============================] - 1s 4ms/step - loss: 2.0945 - accuracy: 0.1361 - val_loss: 1.9926 - val_accuracy: 0.0824
Epoch 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

In [62]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Define the classification function for motion activity
def classify_motion_activity(row):
    activity_encoded = row['activity_encoded']
    if activity_encoded in [13, 20, 21, 3, 6, 18, 4]:
        true_motion = 0
    elif activity_encoded in [6, 18, 4, 13, 20, 21, 3]:
        true_motion = 4
    elif activity_encoded in [1, 7, 8, 15, 6, 18, 4, 13, 20, 21, 3]:
        true_motion = 1
    elif activity_encoded in [11, 17]:
        true_motion = 3
    elif activity_encoded in [0, 10, 14, 23, 2, 16, 22, 12, 19, 5, 9]:
        true_motion = 2
    return true_motion

# Apply the function to create the 'true_motion' column
pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)

# Split the data into features (X) and target variable (y)
X = pivot_data4_merged_filtered[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'true_motion', 'timestamp']]
y = pivot_data4_merged_filtered['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape data for LSTM (samples, timesteps, features)
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

# Define the model
model = Sequential([
    LSTM(units=64, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4_merged_filtered['activity_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Encode class labels for the new target variable
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=20, validation_split=0.2, class_weight=class_weight_dict, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)


/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_2413/2334679296.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['true_motion'] = pivot_data4_merged_filtered.apply(classify_motion_activity, axis=1)


Epoch 1/20
246/246 [==============================] - 5s 7ms/step - loss: 3.1379 - accuracy: 0.1412 - val_loss: 3.0016 - val_accuracy: 0.2091
Epoch 2/20
246/246 [==============================] - 1s 5ms/step - loss: 2.7848 - accuracy: 0.1668 - val_loss: 2.6059 - val_accuracy: 0.2325
Epoch 3/20
246/246 [==============================] - 1s 5ms/step - loss: 2.4108 - accuracy: 0.1440 - val_loss: 2.2718 - val_accuracy: 0.1017
Epoch 4/20
246/246 [==============================] - 1s 5ms/step - loss: 2.3666 - accuracy: 0.1434 - val_loss: 2.2146 - val_accuracy: 0.1109
Epoch 5/20
246/246 [==============================] - 1s 4ms/step - loss: 2.2710 - accuracy: 0.1363 - val_loss: 2.1147 - val_accuracy: 0.1144
Epoch 6/20
246/246 [==============================] - 1s 5ms/step - loss: 2.1692 - accuracy: 0.1433 - val_loss: 2.1057 - val_accuracy: 0.0860
Epoch 7/20
246/246 [==============================] - 1s 6ms/step - loss: 1.9625 - accuracy: 0.1305 - val_loss: 2.0343 - val_accuracy: 0.1165
Epoch 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

The RNN model architecture has the best performance out of the neural networks without class_weights = balanced. But CNN has best performance with class weights. 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.metrics import classification_report

# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'motion_encoded', 'timestamp']]
y = pivot_data4['predicted_activity']

# Convert labels into binary format for multi-label classification
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

from sklearn.multioutput import MultiOutputClassifier

# Initialize a multi-output classifier with GradientBoostingClassifier as the base estimator
multi_gb_model = MultiOutputClassifier(GradientBoostingClassifier())

# Train the model
multi_gb_model.fit(X_train_scaled, y_train)

# Evaluate the model
test_accuracy_gb = multi_gb_model.score(X_test_scaled, y_test)
print("Test Accuracy (Gradient Boosting):", test_accuracy_gb)

# Make predictions
y_pred_gb = multi_gb_model.predict(X_test_scaled)

# Print classification report
print(classification_report(y_test, y_pred_gb, target_names=mlb.classes_.astype(str)))

# Define the file path to save the model
model_file_path = "/Users/emilkoch/Desktop/2Tango/messenger/Research/Model_Save/multi_gb_model.pkl"

# Save the model
joblib.dump(multi_gb_model, model_file_path)

print("Model saved successfully at:", model_file_path)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# Load the previously trained model
model = load_model("/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras")

# Define the new target variable (y) using single true labels
y_single_true_labels = pivot_data4_merged_filtered['activity_encoded']

# Convert labels into binary format for single-label classification
label_binarizer = LabelBinarizer()
y_single_true_binary = label_binarizer.fit_transform(y_single_true_labels)

# Filter X to include only rows with true labels
X_filtered = X.loc[y_single_true_labels.index]

# Split the filtered data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_single_true_binary, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Retrain the model with class weights
history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2, verbose=1, callbacks=[EarlyStopping(patience=5)])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

# Print classification report
print(classification_report(y_test, y_pred, target_names=label_binarizer.classes_.astype(str)))

In [ ]:
model = joblib.load("/Users/emilkoch/Desktop/2Tango/messenger/Research/Model_Save/multi_gb_model.pkl")

# Define the new target variable (y) using single true labels
y_single_true_labels = pivot_data4_merged_filtered['activity_encoded']

# Convert labels into binary format for single-label classification
label_binarizer = LabelBinarizer()
y_single_true_binary = label_binarizer.fit_transform(y_single_true_labels)

# Filter X to include only rows with true labels
X_filtered = X.loc[y_single_true_labels.index]

# Split the filtered data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_single_true_binary, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
multi_gb_model.fit(X_train_scaled, y_train)

# Evaluate the model
test_accuracy_gb = multi_gb_model.score(X_test_scaled, y_test)
print("Test Accuracy (Gradient Boosting):", test_accuracy_gb)

# Make predictions
y_pred_gb = multi_gb_model.predict(X_test_scaled)

# Print classification report
print(classification_report(y_test, y_pred_gb, target_names=mlb.classes_.astype(str)))

# Define the file path to save the model
model_file_path = "/Users/emilkoch/Desktop/2Tango/messenger/Research/Model_Save/multi_gb_model.pkl"

# Save the model
joblib.dump(multi_gb_model, model_file_path)

print("Model saved successfully at:", model_file_path)

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model("/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/motion.keras")

X = pivot_data4_merged_filtered[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]

# Define the new target variable (y) using single true labels
y = pivot_data4_merged_filtered['true_motion']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Encode class labels for the new target variable
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with class weights and early stopping
history = loaded_model.fit(X_train_scaled, y_train_encoded, epochs=100, batch_size= 45, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict, callbacks=[early_stopping])

# Evaluate the retrained model on the test set
test_loss, test_accuracy = loaded_model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = loaded_model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

In [21]:
pivot_data4['motion_encoded']

0              Stationary
1            Other Moving
2              Stationary
3              Stationary
4         Turning/Tilting
               ...       
173657         Stationary
173658         Stationary
173659         Stationary
173660         Stationary
173661         Stationary
Name: motion_encoded, Length: 173662, dtype: object

In [97]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Split the data into features (X) and target variable (y)
X = pivot_data4_merged_filtered[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'timestamp']]
y = pivot_data4_merged_filtered['true_motion']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4_merged_filtered['activity_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Encode class labels for the new target variable
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=20, validation_split=0.2, class_weight=class_weight_dict, callbacks=[early_stopping])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/true_motion.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

Epoch 1/20
246/246 [==============================] - 3s 5ms/step - loss: 2.7252 - accuracy: 0.1330 - val_loss: 1.8795 - val_accuracy: 0.1353
Epoch 2/20
246/246 [==============================] - 1s 4ms/step - loss: 1.7463 - accuracy: 0.2331 - val_loss: 1.3540 - val_accuracy: 0.1450
Epoch 3/20
246/246 [==============================] - 1s 4ms/step - loss: 1.4851 - accuracy: 0.2358 - val_loss: 1.3102 - val_accuracy: 0.1414
Epoch 4/20
246/246 [==============================] - 1s 6ms/step - loss: 1.3126 - accuracy: 0.2135 - val_loss: 1.3098 - val_accuracy: 0.1287
Epoch 5/20
246/246 [==============================] - 1s 4ms/step - loss: 1.2559 - accuracy: 0.1977 - val_loss: 1.2800 - val_accuracy: 0.1328
Epoch 6/20
246/246 [==============================] - 1s 4ms/step - loss: 1.2493 - accuracy: 0.1990 - val_loss: 1.3164 - val_accuracy: 0.1343
Epoch 7/20
246/246 [==============================] - 1s 4ms/step - loss: 1.2378 - accuracy: 0.1744 - val_loss: 1.3327 - val_accuracy: 0.1211
Epoch 

In [101]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

# Load the saved model
model_path = "/Users/emilkoch/Desktop/2Tango/messenger/Research/Model_Save/true_motion.keras"
model = load_model(model_path)

# Preprocess the unlabeled data
X_unlabeled_scaled = scaler.transform(pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'timestamp']])

# Make predictions
y_pred_prob = model.predict(X_unlabeled_scaled)

# Assuming your model outputs probabilities and you want to select the class with the highest probability
y_pred = np.argmax(y_pred_prob, axis=1)

# Add the predicted labels to the DataFrame
pivot_data4['true_motion'] = y_pred


5427/5427 [==============================] - 11s 2ms/step


In [107]:
# Load the saved model
model_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras"
model = load_model(model_path)

# Preprocess the unlabeled data
X_unlabeled_scaled = scaler.transform(pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'true_motion', 'timestamp']])

# Make predictions
y_pred_prob = model.predict(X_unlabeled_scaled)

# Assuming your model outputs probabilities and you want to select the class with the highest probability
y_pred = np.argmax(y_pred_prob, axis=1)

# Add the predicted labels to the DataFrame
pivot_data4['activity_encoded'] = y_pred

5427/5427 [==============================] - 11s 2ms/step


In [108]:
pivot_data4

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,motion_encoded,predicted_activity,true_motion,activity_encoded
0,465,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,3
1,466,0.0,0.0,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Other Moving,1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,15
2,466,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,8
3,466,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,8
4,466,0.0,0.0,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,...,0.0,0.0,0.0,0.0,0.0,Turning/Tilting,3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173657,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,17
173658,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,17
173659,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,17
173660,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Stationary,2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3,17


In [126]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'true_motion', 'timestamp', 'activity_encoded']]
y = pivot_data4['predicted_activity']

# Convert labels into binary format for multi-label classification
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(mlb.classes_), activation='sigmoid')  # Sigmoid activation for multi-label classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=1, callbacks=[EarlyStopping(patience=5)])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/predicted_activities.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Convert predictions back to original labels
y_pred_labels = mlb.inverse_transform(y_pred)

# Convert ground truth labels back to original format
y_true_labels = mlb.inverse_transform(y_test)

# Convert ground truth labels to binary array format
y_true_binary = mlb.transform(y_true_labels)

# Print classification report
print(classification_report(y_true_binary, y_pred, target_names=mlb.classes_.astype(str)))

Epoch 1/10
3474/3474 [==============================] - 16s 4ms/step - loss: 0.1127 - accuracy: 0.0949 - val_loss: 0.0784 - val_accuracy: 0.0188
Epoch 2/10
3474/3474 [==============================] - 14s 4ms/step - loss: 0.0499 - accuracy: 0.0824 - val_loss: 0.0651 - val_accuracy: 0.0075
Epoch 3/10
3474/3474 [==============================] - 15s 4ms/step - loss: 0.0405 - accuracy: 0.0810 - val_loss: 0.0646 - val_accuracy: 0.0032
Epoch 4/10
3474/3474 [==============================] - 13s 4ms/step - loss: 0.0381 - accuracy: 0.0973 - val_loss: 0.0609 - val_accuracy: 0.0081
Epoch 5/10
3474/3474 [==============================] - 14s 4ms/step - loss: 0.0354 - accuracy: 0.0977 - val_loss: 0.0553 - val_accuracy: 0.0071
Epoch 6/10
3474/3474 [==============================] - 13s 4ms/step - loss: 0.0326 - accuracy: 0.0866 - val_loss: 0.0630 - val_accuracy: 0.0127
Epoch 7/10
3474/3474 [==============================] - 14s 4ms/step - loss: 0.0302 - accuracy: 0.0811 - val_loss: 0.0637 - val_ac

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
